In [11]:
import geopandas as gpd
import osmium as osm
import shapely as shp

import pandas as pd

In [2]:
    def node(self, n):    
        self.coords = shp.Point((n.location.x, n.location.y))
        self.tag_inventory(n, "node")
        
        del n

    def way(self, w):
        test = []
        for i in range(len(w.nodes)):
            test.append((w.nodes[i].location.x, w.nodes[i].location.y))
        
        # this only works if we have more than 4 coordinates if not then throw this data entry away
        if (4 <= len(test)):
            self.coords = shp.Polygon(test.copy())
            self.tag_inventory(w, "way")
        
        # if you dont do this the libary sometimes, throws erros, maybe only jupyter issues needs furtehr testing
        del w
        del test

In [3]:
class OSMHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.osm_data = []
        self.coords = 0
        self.tag_list = []
        
    def tag_inventory(self, elem, elem_type):
        self.tag_list = []
        for tag in elem.tags:
            self.tag_list.append((tag.k, tag.v))
        
        if (elem_type == "node"):
            self.osm_data.append([elem_type, 
                                   self.coords,
                                   len(self.tag_list),
                                   self.tag_list])
        elif (elem_type == "way"):
            self.osm_data.append([elem_type, 
                                   0,
                                   len(self.tag_list),
                                   self.tag_list])
        else:
            self.osm_data.append([elem_type, 
                                  self.coords,
                                  len(self.tag_list),
                                  self.tag_list])
            
    
    def node(self, n):    
        self.coords = shp.Point((n.location.x, n.location.y))
        self.tag_inventory(n, "node")
        
        del n

    def way(self, w):
        test = []
        for i in range(len(w.nodes)):
            test.append((w.nodes[i].location.x, w.nodes[i].location.y))
        
        # this only works if we have more than 4 coordinates if not then throw this data entry away
        if (4 <= len(test)):
            self.coords = shp.Polygon(test.copy())
            self.tag_inventory(w, "way")
        
        # if you dont do this the libary sometimes, throws erros, maybe only jupyter issues needs furtehr testing
        del w
        del test

    # its really hard to get the location out of relations
    def relation(self, r):   
        self.tag_inventory(r, "relation")
        
        del r

In [4]:
osmhandler = OSMHandler()
# scan the input file and fills the handler list accordingly
osmhandler.apply_file("../../data/andorra-latest.osm.pbf", locations=True)

In [5]:
# transform the list into a pandas DataFrame
data_colnames = ['osm_type','location', 'ntags', 'tags']
df_osm = gpd.GeoDataFrame(osmhandler.osm_data, columns=data_colnames)
# df_osm = tag_genome.sort_values(by=['type', 'id', 'ts'])

In [6]:
df_osm["Category"] = None

In [7]:
df_osm

,osm_type,location,ntags,tags,Category
0,node,POINT (15527243 425142133),0,[],None
1,node,POINT (15526978 425144526),0,[],None
2,node,POINT (15529780 425152372),0,[],None
3,node,POINT (15535850 425165425),0,[],None
4,node,POINT (15538550 425168939),0,[],None
...,...,...,...,...,...
368936,relation,"POLYGON ((14944460 424913480, 14945762 4249139...",2,"[(landuse, forest), (type, multipolygon)]",None
368937,relation,"POLYGON ((14944460 424913480, 14945762 4249139...",2,"[(landuse, forest), (type, multipolygon)]",None
368938,relation,"POLYGON ((14944460 424913480, 14945762 4249139...",2,"[(landuse, forest), (type, multipolygon)]",None
368939,relation,"POLYGON ((14944460 424913480, 14945762 4249139...",2,"[(landuse, forest), (type, multipolygon)]",None


In [8]:
df_osm.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 368941 entries, 0 to 368940
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   osm_type  368941 non-null  object
 1   location  368941 non-null  object
 2   ntags     368941 non-null  int64 
 3   tags      368941 non-null  object
 4   Category  0 non-null       object
dtypes: int64(1), object(4)
memory usage: 14.1+ MB


## Test Tag filtering

Teste how blacklist and whitelist work in a geodataframe

In [9]:
# defien a whitelist
whitelist = [("Building", "True")]

In [24]:
df_osm["tags"][368940]

[('natural', 'water'), ('type', 'multipolygon'), ('water', 'river')]

In [20]:
# get the tag entry, from the last row
df_osm["tags"][368940]

[('natural', 'water'), ('type', 'multipolygon'), ('water', 'river')]

In [21]:
# get a single tag fro mthe tag entry
df_osm["tags"][368940][0]

('natural', 'water')

In [13]:
df_osm["tags"].loc["Building"]

KeyError: 'Building'

In [13]:
df_osm["tags"].loc["Building"]

KeyError: 'Building'

In [22]:
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.34 ms (started: 2023-01-20 23:04:59 +01:00)


In [27]:
df1=pd.DataFrame({'building1':['true', 'none', 'false', 'true', 'false', 'none'], 'building2':['true', 'none', 'false', 'true', 'false', 'none']})

time: 1.56 ms (started: 2023-01-20 23:07:35 +01:00)


In [28]:
df1

,building1,building2
0,true,true
1,none,none
2,false,false
3,true,true
4,false,false
5,none,none


time: 13.8 ms (started: 2023-01-20 23:07:37 +01:00)


In [30]:
df1["building1"].isin([1])

0    False
1    False
2    False
3    False
4    False
5    False
Name: building1, dtype: bool

time: 7.48 ms (started: 2023-01-20 23:08:22 +01:00)


In [33]:

df1=pd.DataFrame({'tags':[('building1': 'true', 'buidling2': 'true'), 
                          ('building1': 'none', 'buidling2': 'none'), 
                          ('building1': 'false', 'buidling2': 'false'), 
                          ('building1': 'true', 'buidling2': 'true'), 
                          ('building1': 'false', 'buidling2': 'false'), 
                          ('building1': 'none', 'buidling2': 'none')]})

SyntaxError: invalid syntax (1105695677.py, line 1)

time: 2.05 ms (started: 2023-01-20 23:16:52 +01:00)
